<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/318_EaaS_Utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilities for EaaS Orchestrator Agent

In [ ]:
"""Utilities for EaaS Orchestrator Agent

Reusable business logic for data loading, evaluation execution, and scoring.
"""

import json
from pathlib import Path
from typing import Dict, List, Any, Optional
from datetime import datetime


def load_journey_scenarios(data_dir: str, filename: str) -> List[Dict[str, Any]]:
    """Load journey scenarios from JSON file."""
    file_path = Path(data_dir) / filename
    with open(file_path, 'r') as f:
        return json.load(f)


def load_specialist_agents(data_dir: str, filename: str) -> Dict[str, Any]:
    """Load specialist agents configuration from JSON file."""
    file_path = Path(data_dir) / filename
    with open(file_path, 'r') as f:
        return json.load(f)


def load_supporting_data(
    data_dir: str,
    customers_file: str,
    orders_file: str,
    logistics_file: str,
    marketing_signals_file: str
) -> Dict[str, Any]:
    """Load all supporting data files."""
    data = {}

    # Load customers
    customers_path = Path(data_dir) / customers_file
    with open(customers_path, 'r') as f:
        data['customers'] = json.load(f)

    # Load orders
    orders_path = Path(data_dir) / orders_file
    with open(orders_path, 'r') as f:
        data['orders'] = json.load(f)

    # Load logistics
    logistics_path = Path(data_dir) / logistics_file
    with open(logistics_path, 'r') as f:
        data['logistics'] = json.load(f)

    # Load marketing signals
    marketing_path = Path(data_dir) / marketing_signals_file
    with open(marketing_path, 'r') as f:
        data['marketing_signals'] = json.load(f)

    return data


def load_decision_rules(data_dir: str, filename: str) -> Dict[str, Any]:
    """Load orchestrator decision rules from JSON file."""
    file_path = Path(data_dir) / filename
    with open(file_path, 'r') as f:
        content = f.read()
        # The file contains both JSON and Python code, extract just the JSON
        if 'decision_rules_json =' in content:
            # Extract the JSON dictionary
            start = content.find('decision_rules_json = {')
            if start != -1:
                # Find the matching closing brace
                brace_count = 0
                i = start + len('decision_rules_json = ')
                while i < len(content):
                    if content[i] == '{':
                        brace_count += 1
                    elif content[i] == '}':
                        brace_count -= 1
                        if brace_count == 0:
                            json_str = content[start + len('decision_rules_json = '):i+1]
                            return eval(json_str)  # Safe eval for JSON-like dict
        # Fallback: try to parse as JSON
        return json.loads(content)


In [ ]:
def build_agent_lookup(agents: Dict[str, Any]) -> Dict[str, Dict[str, Any]]:
    """Create fast lookup dictionary for agents."""
    return {agent_id: agent_data for agent_id, agent_data in agents.items()}


def build_scenario_lookup(scenarios: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """Create fast lookup dictionary for scenarios."""
    return {s['scenario_id']: s for s in scenarios}


def get_customer_data(customer_id: str, supporting_data: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """Get customer data by ID."""
    customers = supporting_data.get('customers', [])
    return next((c for c in customers if c.get('customer_id') == customer_id), None)


def get_order_data(order_id: str, supporting_data: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """Get order data by ID."""
    orders = supporting_data.get('orders', [])
    return next((o for o in orders if o.get('order_id') == order_id), None)


def get_logistics_data(order_id: str, supporting_data: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """Get logistics data for an order."""
    logistics = supporting_data.get('logistics', {})
    for carrier, orders in logistics.items():
        if order_id in orders:
            return orders[order_id]
    return None


In [ ]:
def simulate_agent_execution(
    agent_id: str,
    scenario: Dict[str, Any],
    supporting_data: Dict[str, Any],
    agents: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Simulate agent execution (MVP: Rule-based simulation).

    In a real implementation, this would call the actual agent.
    For MVP, we simulate based on expected behavior.
    """
    agent = agents.get(agent_id)
    if not agent:
        return {
            "status": "failed",
            "error": f"Agent {agent_id} not found",
            "output": None
        }

    # MVP: Simple rule-based simulation
    # In production, this would be an actual API call to the agent

    # Get relevant data
    customer_id = scenario.get('customer_id')
    order_id = scenario.get('order_id')

    customer = get_customer_data(customer_id, supporting_data)
    order = get_order_data(order_id, supporting_data)
    logistics = get_logistics_data(order_id, supporting_data)

    # Simulate agent response based on agent type
    if agent_id == 'shipping_update_agent':
        output = {
            "status": "shipping_update",
            "carrier": logistics.get('carrier') if logistics else "Unknown",
            "current_status": logistics.get('status') if logistics else "unknown",
            "estimated_delivery": logistics.get('estimated_delivery') if logistics else "Unknown",
            "details": logistics.get('details') if logistics else "No tracking information available"
        }
    elif agent_id == 'refund_agent':
        # Simulate refund calculation
        order = get_order_data(order_id, supporting_data)
        items = order.get('items', []) if order else []
        # Simple refund calculation (in real system, would use actual pricing)
        refund_amount = len(items) * 25.0  # Placeholder
        output = {
            "status": "refund_issued",
            "refund_amount": refund_amount,
            "refunded_at": datetime.now().isoformat(),
            "notes": "Refund processed successfully."
        }
    elif agent_id == 'apology_message_agent':
        output = {
            "status": "apology_message",
            "message": "We're very sorry for the inconvenience with your order. We're taking steps to resolve this as quickly as possible."
        }
    elif agent_id == 'escalation_agent':
        issue_type = scenario.get('expected_issue_type', 'unknown')
        priority = "high" if issue_type in ['lost_package', 'item_not_received'] else "medium"
        output = {
            "status": "escalated",
            "priority": priority,
            "assigned_to": "tier_2_support",
            "notes": "Issue escalated for manual review."
        }
    else:
        output = {
            "status": "unknown",
            "message": f"Agent {agent_id} executed"
        }

    return {
        "status": "completed",
        "output": output,
        "execution_time_seconds": 0.5  # Simulated
    }

In [ ]:
def score_evaluation(
    evaluation: Dict[str, Any],
    expected_outcome: str,
    expected_resolution_path: List[str],
    scoring_weights: Dict[str, float],
    pass_threshold: float
) -> Dict[str, Any]:
    """
    Score an evaluation by comparing actual output to expected.

    MVP: Rule-based scoring. Future: LLM-as-a-judge scoring.
    """
    actual_output = evaluation.get('actual_output', {})
    execution_status = evaluation.get('status', 'failed')

    if execution_status != 'completed':
        return {
            "correctness_score": 0.0,
            "response_time_score": 0.0,
            "output_quality_score": 0.0,
            "overall_score": 0.0,
            "passed": False,
            "issues": [f"Execution failed: {evaluation.get('error', 'Unknown error')}"]
        }

    # Score correctness (matches expected outcome)
    correctness_score = 1.0
    issues = []

    # Check if output status matches expected outcome type
    output_status = actual_output.get('status', '')
    if expected_outcome == 'provide_delivery_update' and output_status != 'shipping_update':
        correctness_score -= 0.3
        issues.append("Output status doesn't match expected outcome type")
    elif expected_outcome == 'issue_refund_and_notify_customer' and output_status != 'refund_issued':
        correctness_score -= 0.3
        issues.append("Output status doesn't match expected outcome type")
    elif expected_outcome == 'acknowledge_delay_and_update_eta' and output_status not in ['shipping_update', 'apology_message']:
        correctness_score -= 0.2
        issues.append("Output doesn't match expected outcome type")

    correctness_score = max(0.0, correctness_score)

    # Score response time
    execution_time = evaluation.get('execution_time_seconds', 10.0)
    response_time_threshold = 2.0  # From config
    if execution_time <= response_time_threshold:
        response_time_score = 1.0
    elif execution_time <= response_time_threshold * 2:
        response_time_score = 0.7
    elif execution_time <= response_time_threshold * 3:
        response_time_score = 0.4
    else:
        response_time_score = 0.1
        issues.append(f"Response time too slow: {execution_time}s")

    # Score output quality (structure/format)
    output_quality_score = 1.0
    if not isinstance(actual_output, dict):
        output_quality_score = 0.0
        issues.append("Output is not a dictionary")
    elif 'status' not in actual_output:
        output_quality_score = 0.5
        issues.append("Output missing 'status' field")
    elif len(actual_output) < 2:
        output_quality_score = 0.7
        issues.append("Output has minimal fields")

    # Calculate overall score
    overall_score = (
        correctness_score * scoring_weights.get('correctness', 0.5) +
        response_time_score * scoring_weights.get('response_time', 0.2) +
        output_quality_score * scoring_weights.get('output_quality', 0.3)
    )

    passed = overall_score >= pass_threshold

    return {
        "correctness_score": correctness_score,
        "response_time_score": response_time_score,
        "output_quality_score": output_quality_score,
        "overall_score": overall_score,
        "passed": passed,
        "issues": issues
    }



In [ ]:
def calculate_agent_performance_summary(
    agent_id: str,
    evaluations: List[Dict[str, Any]],
    scores: List[Dict[str, Any]],
    health_thresholds: Dict[str, float]
) -> Dict[str, Any]:
    """Calculate performance summary for an agent."""
    agent_evaluations = [e for e in evaluations if e.get('target_agent_id') == agent_id]
    agent_scores = [s for s in scores if s.get('target_agent_id') == agent_id]

    if not agent_scores:
        return {
            "agent_id": agent_id,
            "total_evaluations": 0,
            "passed_count": 0,
            "failed_count": 0,
            "average_score": 0.0,
            "average_response_time": 0.0,
            "health_status": "unknown"
        }

    total = len(agent_scores)
    passed = sum(1 for s in agent_scores if s.get('passed', False))
    failed = total - passed
    avg_score = sum(s.get('overall_score', 0.0) for s in agent_scores) / total

    avg_response_time = sum(
        e.get('execution_time_seconds', 0.0) for e in agent_evaluations
    ) / len(agent_evaluations) if agent_evaluations else 0.0

    # Determine health status
    if avg_score >= health_thresholds.get('healthy', 0.85):
        health_status = "healthy"
    elif avg_score >= health_thresholds.get('degraded', 0.70):
        health_status = "degraded"
    else:
        health_status = "critical"

    return {
        "agent_id": agent_id,
        "total_evaluations": total,
        "passed_count": passed,
        "failed_count": failed,
        "average_score": avg_score,
        "average_response_time": avg_response_time,
        "health_status": health_status
    }